<a href="https://colab.research.google.com/github/amoyag/Biofisica/blob/main/S2_NAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Autorregulación negativa en circuitos biológicos**

## Introducción

*Nota: Es conveniente que amplíes este resumen teórico con los apartados 2.4 a 2.6 de An Introduction to Systems Biology de Uri Alon. Además puedes ver este circuito en el artículo [Rosenfeld, N., Elowitz, M. B. & Alon, U. Negative Autoregulation Speeds the Response Times of Transcription Networks. J. Mol. Biol. 323, 785–793 (2002).](https://doi.org/10.1016/S0022-2836(02)00994-4)*

La **autorregulación negativa** (Negative Auto-Regulation, NAR) es un motivo recurrente en sistemas vivos donde un gen regula su propia expresión **inhibiéndose**. En términos simples:

*   El producto del gen (por ejemplo, una proteína reguladora) **reduce la tasa de transcripción** del mismo gen.
*   Este mecanismo introduce una **retroalimentación negativa** que actúa como un control interno.

El modelo matemático típico para este circuito es:


$$\frac{dY}{dt} = \frac{\beta}{1 + (Y/K)^n} - \alpha Y$$

donde:

*   $Y(t)$: concentración del producto génico.
*   $\beta$: tasa máxima de síntesis.
*   $K$: constante de inhibición (nivel de Y donde la síntesis se reduce a la mitad).
*   $n$: coeficiente de Hill (grado de cooperatividad en la inhibición).
*   $\alpha$: tasa de degradación.


## **Propiedades del circuito**

1.  **Aceleración del tiempo de respuesta**
    *   Comparado con un circuito sin autorregulación, el sistema alcanza el estado estacionario **más rápido**.
    *   Esto ocurre porque la retroalimentación negativa evita acumulaciones excesivas y ajusta la producción dinámicamente.

2.  **Reducción de fluctuaciones**
    *   Actúa como un **amortiguador** frente a perturbaciones externas y ruido estocástico.
    *   La inhibición evita que pequeñas variaciones se amplifiquen.

3.  **Robustez frente a cambios en parámetros**
    *   El nivel estacionario de ( Y ) es menos sensible a variaciones en (\beta) o en la entrada externa.
    *   Esto es crucial para mantener la homeostasis celular.

4.  **Estabilidad y control evolutivo**
    *   Motivos de retroalimentación negativa son frecuentes en sistemas biológicos porque:
        *   Mejoran la **precisión temporal**.
        *   Reducen el **costo energético** al evitar sobreproducción.



## **Comparación con circuitos sin autorregulación**

*   **Sin NAR:**  
    
    $$\frac{dY}{dt} = \beta - \alpha Y$$
    Tiempo de respuesta más lento, mayor sensibilidad al ruido.

*   **Con NAR:**  
    
    $$\frac{dY}{dt} = \frac{\beta}{1 + (Y/K)^n} - \alpha Y$$
    Respuesta más rápida, salida más estable.



Para hacer una interpretación intuitiva, piensa en un **termostato biológico**, en el que si la proteína se acumula demasiado, **inhibe su propia producción**, mientras que si hay poca proteína, la inhibición es débil y la síntesis aumenta.

Este mecanismo es un ejemplo de **control adaptativo** en sistemas biológicos.




## Banco de módulos

### 0. Setup inicial
```python

# ============================================
# Módulo 0: Librerías y configuración inicial
# ============================================

# Librerías numéricas y científicas
import numpy as np
from scipy.integrate import solve_ivp

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de gráficos
# seaborn pone fondo gris y fuente grande
#plt.style.use("seaborn-v0_8")
#sns.set_context("talk")
plt.style.use("default")


# Librerías para análisis de datos
import pandas as pd

# Configuración general
%matplotlib inline

# Semilla para reproducibilidad
np.random.seed(42)

print("Entorno listo: NumPy, SciPy, Matplotlib, Seaborn, Pandas importados.")
```


### 1. Modelo del circuito

```python
def model_nar(t, y, params):
    """
    Modelo de autorregulación negativa.
    
    dY/dt = beta / (1 + (Y/K)^n) - alpha * Y
    
    Parámetros:
    - params: diccionario con claves:
        beta: tasa máxima de síntesis
        K: constante de inhibición
        n: coeficiente de Hill
        alpha: tasa de degradación
    """
    Y = y[0]
    beta = params["beta"]
    K = params["K"]
    n = params["n"]
    alpha = params["alpha"]
    
    dYdt = beta / (1 + (Y / K)**n) - alpha * Y
    return [dYdt]


# Consideramos tambien el modelo de regulacion simple

def model_simple(t, y, params):
    """
    Modelo sin autorregulación (regulación simple):
    
    dY/dt = beta - alpha * Y
    
    Parámetros:
    - params: diccionario con claves:
        beta: tasa de síntesis
        alpha: tasa de degradación
    """
    Y = y[0]
    beta = params["beta"]
    alpha = params["alpha"]
    dYdt = beta - alpha * Y
    return [dYdt]


```

### 2. Configuración de la simulación

```python
def build_config(model_params, t_span=(0, 60), num_points=600, y0=[0.0]):
    """
    Construye la configuración para la simulación.
    
    Retorna:
    - params: diccionario con parámetros del modelo
    - y0: condiciones iniciales
    - t_span: tupla (inicio, fin)
    - t_eval: array de tiempos
    """
    import numpy as np
    t_eval = np.linspace(t_span[0], t_span[1], num_points)
    return model_params, y0, t_span, t_eval
```
### 3. Ejecución de la simulación

```python

def run_simulation(model_func, params, y0, t_span, t_eval):
    """
    Ejecuta la simulación usando solve_ivp.
    
    Retorna:
    - sol: objeto con resultados (tiempos y valores)
    """
    sol = solve_ivp(fun=lambda t, y: model_func(t, y, params),
                    t_span=t_span, y0=y0, t_eval=t_eval)
    return sol
```

### 4. Visualización de resultados
```python

def plot_timeseries(sol, labels=None, title="Dinámica temporal"):
    """
    Grafica las series temporales de la simulación con estilo clásico (fondo blanco).
    
    Parámetros:
    - sol: objeto devuelto por solve_ivp (contiene sol.t y sol.y)
    - labels: lista opcional con nombres para cada variable
    - title: título del gráfico
    """
    plt.figure(figsize=(7, 4))
    
    # Iterar sobre todas las variables del sistema
    for i in range(sol.y.shape[0]):
        lab = labels[i] if labels and i < len(labels) else f"var{i}"
        plt.plot(sol.t, sol.y[i], label=lab)
    
    # Etiquetas y título (estilo simple)
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración (a.u.)")
    plt.title(title)
    
    # Leyenda y disposición
    plt.legend()
    plt.tight_layout()
    plt.show()
```
### 5. Cálculo del tiempo de respuesta
```python

def compute_half_time(sol):
    """
    Calcula el tiempo para alcanzar el 50% del estado estacionario.
    """
    Y_final = sol.y[0][-1]
    half_value = 0.5 * Y_final
    # Buscar el primer tiempo donde Y >= half_value
    for t, y in zip(sol.t, sol.y[0]):
        if y >= half_value:
            return t
    return None
```



### 6. Análisis de ruido

```python

def noisy_input(t, sigma=0.2):
    """
    Genera ruido gaussiano para simular entrada fluctuante.
    """
    return np.random.normal(0, sigma)

def analyze_noise_effect(model_func, base_params, y0, t_span, t_eval, sigma_list):
    """
    Compara desviación estándar de salida frente a ruido.
    
    Retorna:
    - dict con sigma: (std_out)
    """
    results = {}
    for sigma in sigma_list:
        Y_values = []
        current_y = y0[0]
        for t in t_eval:
            # Actualizar beta con ruido
            params = base_params.copy()
            params["beta"] += noisy_input(t, sigma)
            # Paso de integración simple (Euler)
            dYdt = model_func(t, [current_y], params)[0]
            current_y += dYdt * (t_eval[1] - t_eval[0])
            Y_values.append(current_y)
        results[sigma] = np.std(Y_values)
    return results
```

### 7. Barrido de parámetros y mapa de calor

```python

def compute_steady_state(model_func, params, y0, t_span, t_eval):
    """
    Ejecuta simulación y retorna el valor estacionario.
    """
    sol = run_simulation(model_func, params, y0, t_span, t_eval)
    return sol.y[0][-1]

def parameter_scan(model_func, beta_range, alpha_range, fixed_params, y0, t_span, t_eval):
    """
    Barrido de parámetros beta y alpha.
    
    Retorna:
    - matriz con valores estacionarios
    """
    results = np.zeros((len(beta_range), len(alpha_range)))
    for i, beta in enumerate(beta_range):
        for j, alpha in enumerate(alpha_range):
            params = fixed_params.copy()
            params["beta"] = beta
            params["alpha"] = alpha
            results[i, j] = compute_steady_state(model_func, params, y0, t_span, t_eval)
    return results
```


In [ ]:

#@title Módulo 0: Librerías y configuración inicial
# ============================================

# Librerías numéricas y científicas
import numpy as np
from scipy.integrate import solve_ivp

# Librerías para visualización
import matplotlib.pyplot as plt
#import seaborn as sns

# Configuración de gráficos
#plt.style.use("seaborn-v0_8")
#sns.set_context("talk")
plt.style.use("default")



# Librerías para análisis de datos
import pandas as pd

# Configuración general
%matplotlib inline

# Semilla para reproducibilidad
np.random.seed(42)

#print("Entorno listo: NumPy, SciPy, Matplotlib, Seaborn, Pandas importados.")

In [ ]:
#@title Módulo 1: Modelo del circuito
def model_nar(t, y, params):
    """
    Modelo de autorregulación negativa.

    dY/dt = beta / (1 + (Y/K)^n) - alpha * Y

    Parámetros:
    - params: diccionario con claves:
        beta: tasa máxima de síntesis
        K: constante de inhibición
        n: coeficiente de Hill
        alpha: tasa de degradación
    """
    Y = y[0]
    beta = params["beta"]
    K = params["K"]
    n = params["n"]
    alpha = params["alpha"]

    dYdt = beta / (1 + (Y / K)**n) - alpha * Y
    return [dYdt]



def model_simple(t, y, params):
    """
    Modelo sin autorregulación (regulación simple):

    dY/dt = beta - alpha * Y

    Parámetros:
    - params: diccionario con claves:
        beta: tasa de síntesis
        alpha: tasa de degradación
    """
    Y = y[0]
    beta = params["beta"]
    alpha = params["alpha"]
    dYdt = beta - alpha * Y
    return [dYdt]


In [ ]:
#@title Módulo 2: Configuracion de la simulación
def build_config(model_params, t_span=(0, 60), num_points=600, y0=[0.0]):
    """
    Construye la configuración para la simulación.

    Retorna:
    - params: diccionario con parámetros del modelo
    - y0: condiciones iniciales
    - t_span: tupla (inicio, fin)
    - t_eval: array de tiempos
    """
    import numpy as np
    t_eval = np.linspace(t_span[0], t_span[1], num_points)
    return model_params, y0, t_span, t_eval



In [ ]:
#@title Módulo 3: Eejecución de la simulación
def run_simulation(model_func, params, y0, t_span, t_eval):
    """
    Ejecuta la simulación usando solve_ivp.

    Retorna:
    - sol: objeto con resultados (tiempos y valores)
    """
    sol = solve_ivp(fun=lambda t, y: model_func(t, y, params),
                    t_span=t_span, y0=y0, t_eval=t_eval)
    return sol

In [ ]:
#@title Módulo 4: Visualización


def plot_timeseries(sol, labels=None, title="Dinámica temporal"):
    """
    Grafica las series temporales de la simulación con estilo clásico (fondo blanco).

    Parámetros:
    - sol: objeto devuelto por solve_ivp (contiene sol.t y sol.y)
    - labels: lista opcional con nombres para cada variable
    - title: título del gráfico
    """
    plt.figure(figsize=(7, 4))

    # Iterar sobre todas las variables del sistema
    for i in range(sol.y.shape[0]):
        lab = labels[i] if labels and i < len(labels) else f"var{i}"
        plt.plot(sol.t, sol.y[i], label=lab)

    # Etiquetas y título (estilo simple)
    plt.xlabel("Tiempo")
    plt.ylabel("Concentración (a.u.)")
    plt.title(title)

    # Leyenda y disposición
    plt.legend()
    plt.tight_layout()
    plt.show()





In [ ]:
#@title Módulo 5: tiempo de respuesta
def compute_half_time(sol):
    """
    Calcula el tiempo para alcanzar el 50% del estado estacionario.
    """
    Y_final = sol.y[0][-1]
    half_value = 0.5 * Y_final
    # Buscar el primer tiempo donde Y >= half_value
    for t, y in zip(sol.t, sol.y[0]):
        if y >= half_value:
            return t
    return None

In [ ]:
#@title Módulo 6: Análisis de ruido
def noisy_input(t, sigma=0.2):
    """
    Genera ruido gaussiano para simular entrada fluctuante.
    """
    return np.random.normal(0, sigma)

def analyze_noise_effect(model_func, base_params, y0, t_span, t_eval, sigma_list):
    """
    Compara desviación estándar de salida frente a ruido.

    Retorna:
    - dict con sigma: (std_out)
    """
    results = {}
    for sigma in sigma_list:
        Y_values = []
        current_y = y0[0]
        for t in t_eval:
            # Actualizar beta con ruido
            params = base_params.copy()
            params["beta"] += noisy_input(t, sigma)
            # Paso de integración simple (Euler)
            dYdt = model_func(t, [current_y], params)[0]
            current_y += dYdt * (t_eval[1] - t_eval[0])
            Y_values.append(current_y)
        results[sigma] = np.std(Y_values)
    return results


In [ ]:
#@title Módulo 7: Barrido de parámetros
def compute_steady_state(model_func, params, y0, t_span, t_eval):
    """
    Ejecuta simulación y retorna el valor estacionario.
    """
    sol = run_simulation(model_func, params, y0, t_span, t_eval)
    return sol.y[0][-1]

def parameter_scan(model_func, beta_range, alpha_range, fixed_params, y0, t_span, t_eval):
    """
    Barrido de parámetros beta y alpha.

    Retorna:
    - matriz con valores estacionarios
    """
    results = np.zeros((len(beta_range), len(alpha_range)))
    for i, beta in enumerate(beta_range):
        for j, alpha in enumerate(alpha_range):
            params = fixed_params.copy()
            params["beta"] = beta
            params["alpha"] = alpha
            results[i, j] = compute_steady_state(model_func, params, y0, t_span, t_eval)
    return results


## Ejemplo de ensamblado de módulos y preparación de una simulación

En la celda que tienes a continuación se muestra cómo ensamblar los módulos en una simulación sencilla que ejecuta el modelo y calcula el tiempo de respuesta.

In [ ]:
#@title Simulación genérica de NAR
# ============================================

# 1. Definir parámetros del modelo
params_nar = {
    "beta": 1.0,   # tasa máxima de síntesis
    "alpha": 0.05, # tasa de degradación
    "K": 0.5,      # constante de inhibición
    "n": 2         # coeficiente de Hill
}

# 2. Construir configuración de simulación
model_params, y0, t_span, t_eval = build_config(params_nar, t_span=(0, 60), num_points=600, y0=[0.0])

# 3. Ejecutar la simulación
sol_nar = run_simulation(model_nar, model_params, y0, t_span, t_eval)

# 4. Visualizar la dinámica
plot_timeseries(sol_nar, labels="NAR")

# 5. Calcular tiempo de respuesta (t_1/2)
t_half = compute_half_time(sol_nar)
print(f"Tiempo para alcanzar el 50% del estado estacionario: {t_half:.2f} unidades de tiempo")


## Ejercicios

### Ejercicio 1: Comparación del tiempo de respuesta con y sin autorregulación negativa
Objetivo:
Comprobar que la autorregulación negativa acelera la respuesta del sistema en comparación con un circuito sin autorregulación.
Instrucciones:

Simula dos circuitos:

Circuito simple: sin autorregulación (usa model_simple).
Circuito con NAR: autorregulación negativa (usa model_nar).


Usa los mismos parámetros básicos para ambos modelos:

$\beta = 1.0, \alpha = 0.05$. Para NAR añade: $K = 0.5, n = 2$.


Calcula el tiempo para alcanzar el 50% del estado estacionario $t_{\frac{1}{2}}$ en ambos casos.
Representa las curvas temporales en un mismo gráfico y compara visualmente.
Pregunta: ¿qué sistema responde más rápido y por qué?

Utiliza esta plantilla para preparar tu simulación:

---


```python

# ============================================
# EJERCICIO 1: Comparación del tiempo de respuesta
# ============================================

# 1. Definir parámetros para ambos modelos
params_simple = {...}
params_nar = {...}

# 2. Construir configuración común
model_params_simple, y0, t_span, t_eval = build_config(...)
model_params_nar, _, _, _ = build_config(...)

# 3. Ejecutar simulaciones
sol_simple = run_simulation(model_simple, ...)
sol_nar = run_simulation(model_nar, ...)

# 4. Visualizar resultados (puedes usar plot_timeseries o un gráfico combinado)
plot_timeseries(...)
plot_timeseries(...)

# 5. Calcular tiempos de respuesta
t_half_simple = compute_half_time(...)
t_half_nar = compute_half_time(...)
print(f"Tiempo para alcanzar 50% del estado estacionario:")
print(f" - Sin autorregulación: {...:.2f}")
print(f" - Con autorregulación negativa: {...:.2f}")
```

### Ejercicio 2:  Robustez frente a ruido en la entrada

Objetivo:
Evaluar si la autorregulación negativa reduce la variabilidad frente a entradas ruidosas en comparación con un circuito sin autorregulación.

Instrucciones:

1. Define los parámetros base para ambos modelos:

  - Circuito simple: $\beta, \alpha$.
  - Circuito con NAR: añade $K$ y $n$.

2. Introduce una señal externa con ruido gaussiano (usa el módulo noisy_input).

3. Simula ambos circuitos bajo condiciones ruidosas.
4. Calcula la desviación estándar de la salida en fase estacionaria para cada circuito.
5. Compara los resultados y responde: ¿Cuál circuito filtra mejor el ruido?

Usa esta plantilla para montar tu simulación

```python

# ============================================
# EJERCICIO 3: Robustez frente a ruido
# ============================================

# 1. Definir parámetros base para ambos modelos
params_simple = {...}
params_nar = {...}

# 2. Configuración de simulación
model_params_simple, y0, t_span, t_eval = build_config(...)
model_params_nar, _, _, _ = build_config(...)

# 3. Definir niveles de ruido (sigma)
sigma_list = [...]

# 4. Analizar efecto del ruido en ambos modelos
results_simple = analyze_noise_effect(model_simple, ...)
results_nar = analyze_noise_effect(model_nar, ...)

# 5. Mostrar resultados

for sigma in sigma_list:
    std_simple = results_simple[sigma]
    std_nar = results_nar[sigma]
    print(f"Ruido sigma={sigma}: Simple={std_simple:.4f}, NAR={std_nar:.4f}")

```


### Ejercicio 3: Barrido de parámetros y estabilidad del estado estacionario
Objetivo:
Explorar cómo cambia el nivel estacionario del circuito con autorregulación negativa al variar los parámetros $\beta$ (tasa de síntesis) y $\alpha$ (tasa de degradación). Representar los resultados en un mapa de calor.

Instrucciones:

1. Fija los parámetros $K$ y $n$ (por ejemplo, $K = 0.5, n = 2$).

2. Define rangos para:
  $\beta$ en [0.5, 2.0] (ej. 10 valores).
  $\alpha$ en [0.01, 0.1] (ej. 10 valores).


3. Para cada combinación ($\beta$, $\alpha$):
  - Simula el circuito con NAR
  - Calcula el valor estacionario (último punto de la simulación)


4. Guarda los resultados en una matriz.
5. Representa la matriz como un mapa de calor (ejes: $\beta$ y $\alpha$).


Pregunta: ¿El sistema es más robusto con NAR frente a cambios en $\beta$ y $\alpha$?

Puedes usar esta plantilla para preparar tu simulación:

```python

# ============================================
# EJERCICIO 3: Barrido de parámetros y mapa de calor
# ============================================

# 1. Fijar parámetros constantes (K, n)
fixed_params = {...}

# 2. Definir rangos para beta y alpha
beta_range = np.linspace(...)
alpha_range = np.linspace(...)

# 3. Configuración común
y0, t_span, t_eval = ...

# 4. Crear matriz para resultados
results = np.zeros((len(beta_range), len(alpha_range)))

# 5. Bucle doble para simular cada combinación
for i, beta in enumerate(beta_range):
    for j, alpha in enumerate(alpha_range):
        # Actualizar parámetros
        # Ejecutar simulación con run_simulation
        # Guardar valor estacionario en results[i, j]

# 6. Representar mapa de calor con matplotlib o seaborn
plt.figure(...)
sns.heatmap(...)
plt.xlabel(...)
plt.ylabel(...)
plt.title(...)
plt.show()

```
